In [1]:
import pandas as pd
import pyarrow.parquet as pq
from datetime import datetime

# 读取数据
price_vol_df = pd.read_parquet(r"D:\workspace\xiaoyao\data\stock_daily_price.parquet")  # 量价数据
auction_df = pd.read_parquet(r"D:\workspace\xiaoyao\data\stock_daily_auction.parquet")    # 竞价数据
industry_df = pd.read_parquet(r"D:\workspace\xiaoyao\data\stock_daily_industry.parquet")  # 行业数据

In [2]:
# 统一日期格式（假设原日期是字符串或带时区的格式）
price_vol_df["date"] = pd.to_datetime(price_vol_df["date"]).dt.date
auction_df["date"] = pd.to_datetime(auction_df["date"]).dt.date
industry_df["date"] = pd.to_datetime(industry_df["date"]).dt.date

# 日期区间过滤
start_date = datetime(2025, 1, 1).date()
end_date = datetime(2025, 9, 25).date()

# 去除停牌的股票 ，仅保留price_vol_df 里 paused==0 的
price_vol_df = price_vol_df[(price_vol_df["date"] >= start_date) & (price_vol_df["date"] <= end_date) & (price_vol_df["paused"] == 0)]
auction_df = auction_df[(auction_df["date"] >= start_date) & (auction_df["date"] <= end_date)]
industry_df = industry_df[(industry_df["date"] >= start_date) & (industry_df["date"] <= end_date)]

In [4]:
# 竞价数据字段重命名（示例：前缀au_）
auction_df = auction_df.rename(columns={
    "volume": "au_volume", 
    "money": "au_money",
})

In [5]:
def calculate_auction_indicators(df):
    """计算竞价相关衍生指标"""
    # 竞价量比 = 竞价成交量 / 最近N日平均竞价成交量（示例N=5，不包含当日，取最近N-1日）
    df["auction_volume_60d_ratio"] = df["au_volume"] / df.groupby("stock_code")["au_volume"].rolling(60, closed="left").mean().reset_index(0, drop=True)
    df["auction_volume_5d_ratio"] = df["au_volume"] / df.groupby("stock_code")["au_volume"].rolling(5, closed="left").mean().reset_index(0, drop=True)
    df["auction_volume_1d_ratio"] = df["au_volume"] / df.groupby("stock_code")["au_volume"].rolling(1, closed="left").mean().reset_index(0, drop=True)


    # 买一量 / 卖一量（假设买一量是a1_v，卖一量是b1_v）
    df["bid1_ask1_ratio"] = df["a1_v"] / df["b1_v"]
    
    # 五档买量和 / 五档卖量和（假设买一至买五量是a1_v~a5_v，卖一至卖五量是b1_v~b5_v）
    df["total_bid_vol"] = df["a1_v"] + df["a2_v"] + df["a3_v"] + df["a4_v"] + df["a5_v"]
    df["total_ask_vol"] = df["b1_v"] + df["b2_v"] + df["b3_v"] + df["b4_v"] + df["b5_v"]
    df["bid_ask_ratio"] = df["total_bid_vol"] / df["total_ask_vol"]
    
    return df

# 计算竞价衍生指标
auction_with_indicators = calculate_auction_indicators(auction_df)

# 仅保留需要的字段 auction_volume_60d_ratio auction_volume_5d_ratio auction_volume_1d_ratio bid1_ask1_ratio bid_ask_ratio
auction_with_indicators = auction_with_indicators[
    [
        "date",
        "stock_code",
        "au_volume",
        "au_money",
        "auction_volume_60d_ratio",
        "auction_volume_5d_ratio",
        "auction_volume_1d_ratio",
        "bid1_ask1_ratio",
        "bid_ask_ratio"
    ]
]
auction_with_indicators

,date,stock_code,au_volume,au_money,auction_volume_60d_ratio,auction_volume_5d_ratio,auction_volume_1d_ratio,bid1_ask1_ratio,bid_ask_ratio
5593783,2025-01-02,000001.XSHE,583500.0,6844455.00,NaN,NaN,NaN,1.859621,0.903431
5593784,2025-01-02,000002.XSHE,556110.0,4031797.50,NaN,NaN,NaN,0.588350,0.567159
5593785,2025-01-02,000004.XSHE,70300.0,965219.00,NaN,NaN,NaN,0.035714,0.383495
5593786,2025-01-02,000006.XSHE,275200.0,1989696.00,NaN,NaN,NaN,2.744479,0.895698
5593787,2025-01-02,000007.XSHE,9300.0,65100.00,NaN,NaN,NaN,0.250000,1.620192
...,...,...,...,...,...,...,...,...,...
6517098,2025-09-25,688716.XSHG,18973.0,880347.20,0.341929,0.553139,0.977587,1.594896,0.161939
6517099,2025-09-25,688726.XSHG,2150.0,100383.50,0.154836,0.295614,0.122396,0.405455,0.660330
6517100,2025-09-25,688750.XSHG,10774.0,228301.06,0.415922,1.911572,2.065964,12.162162,1.275258
6517101,2025-09-25,688755.XSHG,5980.0,274003.60,0.792986,2.399872,inf,0.147606,0.273996


In [6]:
import pandas as pd
import numpy as np

def calculate_and_filter_quality_stocks(price_vol_df):
    """
    精简版指标计算：仅保留MA5 + 新增BOLL(OPEN,6)，并筛选基础优质股票
    输入：包含date, stock_code, open, high, low, close, volume, high_limit的量价数据
    输出：
        - indicators_df: 含MA5、BOLL(OPEN,6)指标的完整数据
        - quality_stocks: 基于MA5和BOLL筛选的优质股票数据
    """
    # 复制数据并按【股票代码+日期】排序（确保时间顺序，避免跨股票计算错误）
    df = price_vol_df.copy()
    # 先按股票分组，再按日期排序，保证同一只股票内时间连续
    df = df.sort_values(['stock_code', 'date']).reset_index(drop=True)
    grouped = df.groupby('stock_code')  # 按股票分组计算指标


    # ----------------------
    # 核心指标计算（仅2类：MA5 + BOLL(OPEN,6)）
    # ----------------------
    # 1. 短期移动平均线 MA5（基于前5日收盘价，不含当日，避免未来函数）
    # 逻辑：当日MA5 = 前1日至前5日收盘价的均值（确保计算时不使用当日收盘价）
    df['MA5'] = grouped['close'].shift(1).rolling(
        window=5,  # 5日周期
        min_periods=5,  # 至少有5个数据才计算（避免初期数据不足导致的偏差）
        closed='right'  # 窗口包含右边界（即前1日、前2日...前5日）
    ).mean().reset_index(level=0, drop=True)  # 重置分组索引，避免冗余


    # 2. BOLL指标（基于开盘价，周期6，即 BOLL(OPEN,6)）
    # 公式：
    # - 中轨(mid) = N日开盘价移动平均（此处N=6，包含当日开盘价）
    # - 标准差(std) = N日开盘价的滚动标准差（包含当日开盘价）
    # - 上轨(up) = 中轨 + 2*标准差（常用2倍标准差，覆盖约95%波动范围）
    # - 下轨(low) = 中轨 - 2*标准差
    # 注：BOLL使用当日开盘价参与计算，符合实时交易时的可用数据场景
    
    # 2.1 中轨（mid）：6日开盘价移动平均（包含当日open）
    df['boll_mid'] = grouped['open'].rolling(
        window=6,
        min_periods=6,  # 至少6个数据才计算（确保周期完整）
        closed='right'
    ).mean().reset_index(level=0, drop=True)

    # 2.2 6日开盘价滚动标准差（包含当日open）
    df['boll_std'] = grouped['open'].rolling(
        window=6,
        min_periods=6,
        closed='right'
    ).std().reset_index(level=0, drop=True)  # 标准差用于计算上下轨

    # 2.3 上轨（up）：中轨 + 2*标准差
    df['boll_up'] = df['boll_mid'] + 2 * df['boll_std']

    # 2.4 下轨（low）：中轨 - 2*标准差
    df['boll_low'] = df['boll_mid'] - 2 * df['boll_std']


    # ----------------------
    # 优质股票筛选（基于MA5 + BOLL逻辑，聚焦“趋势+波动安全”）
    # ----------------------
    # 筛选核心：价格在MA5上方（短期趋势向上） + 开盘价在BOLL中轨上方（波动区间内强势）
    quality_stocks = df[
        # 1. MA5相关：排除MA5为空（数据不足） + 开盘价在MA5上方（短期趋势向好）
        (df['MA5'].notna()) &  
        (df['open'] > df['MA5']) &  

        # 2. BOLL相关：排除BOLL指标为空（数据不足） + 开盘价在中轨上方（波动区间内强势）
        (df['boll_mid'].notna()) &  
        (df['open'] > df['boll_mid']) &  

        # 3. 基础安全条件：排除开盘涨停（无买入机会） + 排除停牌（数据异常）
        (df['open'] != df['high_limit']) &  
        (df['volume'] > 0)  # 成交量>0，确保股票有流动性
    ].copy()


    # （可选）按“BOLL强势度”排序，取每日前20%最优质标的
    if not quality_stocks.empty:
        # 计算BOLL强势度：(当日开盘价 - BOLL中轨) / BOLL中轨 → 正值越大，强势度越高
        quality_stocks['boll_strength'] = (quality_stocks['open'] - quality_stocks['boll_mid']) / quality_stocks['boll_mid']
        
        # 按日期分组，每组取强势度前20%的标的（进一步聚焦高弹性）
        quality_stocks = quality_stocks.groupby('date').apply(
            lambda x: x.sort_values('boll_strength', ascending=False).head(
                int(len(x) * 0.2)  # 前20%
            )
        ).reset_index(drop=True)


    # 清理异常值（替换无穷大/负无穷大为空值，避免后续计算报错）
    indicators_df = df.replace([np.inf, -np.inf], np.nan)
    quality_stocks = quality_stocks.replace([np.inf, -np.inf], np.nan)

    return indicators_df.reset_index(drop=True), quality_stocks

# 使用示例
if __name__ == "__main__":
    all_indicators, quality_stocks = calculate_and_filter_quality_stocks(price_vol_df)

C:\Users\jay\AppData\Local\Temp\ipykernel_34420\595035938.py:85: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  quality_stocks = quality_stocks.groupby('date').apply(


In [7]:
# 1. 关联量价（带指标）和竞价（带指标）：按date和stock_code
merged_df = pd.merge(
    all_indicators, 
    auction_with_indicators, 
    on=["date", "stock_code"], 
    how="inner"  # 取两者都有的日期和股票
)

# 2. 关联行业数据：按date和stock_code（行业数据可能是每日或历史快照，需确保关联逻辑）
final_df = pd.merge(
    merged_df, 
    industry_df[["date", "stock_code", "jq_l1_industry_name", "jq_l2_industry_name", "sw_l1_industry_name", "sw_l2_industry_name", "sw_l3_industry_name"]], 
    on=["date", "stock_code"], 
    how="left"  # 行业数据若缺失，填充为NaN
)

In [8]:
def calculate_future_returns(df):
    """计算未来N日收益率"""
    # 当日收盘价
    df["return_dt"] = (df["close"] - df["open"]) / df["close"] * 100

    # 未来1日收盘价（shift(-1)实现）
    df["close_next1"] = df.groupby("stock_code")["close"].shift(-1)
    df["return_next1"] = (df["close_next1"] - df["open"]) / df["open"] * 100

    # 未来3日收盘价（shift(-3)实现）
    df["close_next3"] = df.groupby("stock_code")["close"].shift(-3)
    df["return_next3"] = (df["close_next3"] - df["open"]) / df["open"] * 100

    return df

final_df = calculate_future_returns(final_df)

In [9]:
# 保存为Parquet（压缩存储，适合大数据）
final_df.to_parquet("wide.parquet", index=False)

In [ ]:
import pandas as pd
import numpy as np

def process_daily_stocks_manual(
    df,
    # 基础筛选参数（核心：放宽行业与量比，提高选股覆盖度）
    top_stock_pct=0.1,          # 按量比取前10%（保留，高量能是核心）
    min_industry_count=3,       # 热门行业门槛：从5→3（更灵活，捕捉小热门赛道）
    top_industry_num=3,         # 选取3个热门行业（聚焦核心，不分散）
    final_stock_num=4,          # 最终4只标的（平衡收益与风险）
    min_final_stock=1,          # 最低1只标的（避免无标的可买）
    
    # 技术指标筛选（仅保留MA5，其他全部删除）
    price_above_ma5_required=True, # 仅保留：开盘价在MA5上方（短期趋势信号）
    ma_bullish_required=False,   # 关闭：MA5>MA20（避免过滤掉短期强势股）
    
    # 竞价指标（大幅放宽量比，扩大选股池）
    auction_ratio_col='auction_volume_5d_ratio',  # 5日量比（稳定）
    auction_ratio_min=1.0,      # 量比门槛：从1.2→1.0（最大程度放宽）
    
    return_cumulative=False     # 是否返回累计收益
):
    """
    复现早上高收益逻辑：仅用MA5过滤+宽量比+灵活行业，无其他技术门槛
    核心：少过滤、多覆盖，捕捉量能+短期趋势的双重信号
    """
    # 确保日期格式正确
    if not pd.api.types.is_datetime64_any_dtype(df['date']):
        df['date'] = pd.to_datetime(df['date'])
    
    # 按自然日分组
    df['trade_date'] = df['date'].dt.date
    all_dates = sorted(df['trade_date'].unique())
    
    daily_results = []
    daily_buy_lists = []
    
    for trade_date in all_dates:
        # ---------------------- 步骤1：基础过滤 + 按量比取前10% ----------------------
        daily_data = df[df['trade_date'] == trade_date].copy()
        if daily_data.empty:
            continue
        
        # 基础过滤：仅保留“非停牌、非科创板、非开盘涨停、量比≥1.0、收益率非空”
        base_filtered = daily_data[
            (daily_data['paused'] == 0) &  # 非停牌
            (~daily_data['stock_code'].str.startswith('688')) &  # 非科创板
            (daily_data['open'] != daily_data['high_limit']) &  # 非开盘涨停（有买入机会）
            (daily_data[auction_ratio_col] >= auction_ratio_min) &  # 量比≥1.0（放宽）
            (daily_data['return_next1'].notna())  # 过滤收益率NaN，避免计算错误
        ]
        if len(base_filtered) == 0:
            continue
        
        # 按量比降序，取前10%
        top_pct_count = max(1, int(len(base_filtered) * top_stock_pct))
        sorted_by_ratio = base_filtered.sort_values(auction_ratio_col, ascending=False)
        top_ratio_stocks = sorted_by_ratio.head(top_pct_count)
        
        # ---------------------- 步骤2：选热门行业 ----------------------
        # 统计top10%股票的行业分布（sw_l2行业，更精准）
        industry_counts = top_ratio_stocks['sw_l3_industry_name'].value_counts()
        # 热门行业：至少3只股票，取前3个
        hot_industries = industry_counts[industry_counts >= min_industry_count].sort_values(ascending=False)
        selected_industries = hot_industries.index.tolist()[:top_industry_num] if len(hot_industries)>=top_industry_num else hot_industries.index.tolist()
        if not selected_industries:
            continue
        
        # 筛选热门行业内的股票
        hot_industry_stocks = top_ratio_stocks[top_ratio_stocks['sw_l3_industry_name'].isin(selected_industries)]
        if len(hot_industry_stocks) == 0:
            continue
        
        # ---------------------- 步骤3：仅用MA5过滤（核心简化） ----------------------
        tech_filtered = hot_industry_stocks.copy()
        # # 仅保留“开盘价在MA5上方”，其他过滤全部删除
        # if price_above_ma5_required:
        #     tech_filtered = tech_filtered[tech_filtered['price_above_ma5'] == 1]
        # if len(tech_filtered) == 0:
        #     continue
        
        tech_filtered = tech_filtered[tech_filtered['open'] < tech_filtered['boll_up'] ]
        # tech_filtered = tech_filtered[tech_filtered['open'] > tech_filtered['MA5'] ]

        
        # ---------------------- 步骤4：按量比再排序，取前4名 ----------------------
        final_sorted = tech_filtered.sort_values(auction_ratio_col, ascending=False)
        if len(final_sorted) < min_final_stock:
            continue
        final_stocks = final_sorted.head(final_stock_num)
        
        # ---------------------- 记录结果 ----------------------
        # 计算当日平均收益率（用return_next1，核心收益来源）
        avg_returns = {
            'trade_date': trade_date,
            'avg_return_next1': final_stocks['return_next1'].mean(),
            'stock_count': len(final_stocks),
            'hot_industries': ', '.join(selected_industries)
        }
        daily_results.append(avg_returns)
        
        # 记录买入清单
        buy_list = {
            'trade_date': trade_date,
            'stock_codes': final_stocks['stock_code'].tolist(),
            'auction_ratios': final_stocks[auction_ratio_col].tolist(),
            'industries': final_stocks['sw_l3_industry_name'].tolist()
        }
        daily_buy_lists.append(buy_list)
    
    # ---------------------- 计算累计收益（修复NaN问题） ----------------------
    result_df = pd.DataFrame(daily_results)
    buy_list_df = pd.DataFrame(daily_buy_lists)
    
    if not result_df.empty:
        # 按日期排序，过滤收益率NaN（避免累计收益中断）
        result_df = result_df.sort_values('trade_date').dropna(subset=['avg_return_next1'])
        # 50%仓位（平衡风险，若想更高收益可改0.7）
        result_df['daily_factor'] = 1 + 0.5 * result_df['avg_return_next1'] / 100
        result_df['cum_return'] = result_df['daily_factor'].cumprod()
        # 最终累计收益率（取最后一条有效数据）
        final_return = result_df['cum_return'].iloc[-1] if len(result_df) > 0 else np.nan
    else:
        final_return = np.nan
        result_df['cum_return'] = []
    
    if return_cumulative:
        return final_return, result_df, buy_list_df
    else:
        return final_return

# ---------------------- 主程序（复现高收益参数） ----------------------
if __name__ == "__main__":
    # 1. 加载数据（确保包含：sw_l3_industry_name、price_above_ma5、return_next1）
    df = pd.read_parquet("wide.parquet")
    
    # 2. 复现早上高收益的参数配置（无多余过滤，仅MA5+宽量比）
    params = {
        # 步骤1：量比参数
        'top_stock_pct': 0.1,
        'auction_ratio_col': 'auction_volume_5d_ratio',
        'auction_ratio_min': 1.0,  # 最宽量比门槛
        
        # 步骤2：行业参数
        'min_industry_count': 5,   # 灵活行业门槛
        'top_industry_num': 5,
        
        # 步骤4：标的数量
        'final_stock_num': 4,
        'min_final_stock': 4
    }
    
    # 3. 运行策略
    final_return, result_df, buy_list_df = process_daily_stocks_manual(
        df, 
        **params,
        return_cumulative=True
    )
    
    # 4. 输出结果（重点看cum_return是否≥1.9，即90%+正收益）
    print("="*60)
    print(f"策略最终累计收益率: {final_return:.4f}（目标：≥1.9即90%+正收益）")
    print("="*60)
    print("\n最近20日收益率与累计收益（核心验证）:")
    print(result_df[['trade_date', 'avg_return_next1', 'cum_return', 'hot_industries', 'stock_count']].tail(20))
    print("\n最近5日买入清单（验证标的质量）:")
    for idx, row in buy_list_df.tail(5).iterrows():
        print(f"\n{row['trade_date']} 买入标的:")
        for code, ratio, industry in zip(row['stock_codes'], row['auction_ratios'], row['industries']):
            print(f"  股票代码: {code} | 5日量比: {ratio:.2f} | 行业: {industry}")
    
    # 5. 保存结果（便于后续分析）
    result_df.to_csv("high_return_reproduce.csv", index=False, encoding='utf-8')
    buy_list_df.to_csv("high_return_buy_list.csv", index=False, encoding='utf-8')
    print("\n\n复现结果已保存至:")
    print(" - high_return_reproduce.csv: 每日收益率+累计收益（核心验证文件）")
    print(" - high_return_buy_list.csv: 每日买入清单")